In [2]:
from all_history import *

['C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Consultant\\AppData\\Local\\Programs\\Python\\Python310', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310', '', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages\\win32', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages\\win32\\lib', 'G:\\My Drive\\Software\\PycharmProjects\\tp\\tp_310\\lib\\site-packages\\Pythonwin']
G:\My Drive\vepar\output-May-15.xlsx


In [3]:
common_include.unittest = True

In [4]:
h = Historys()

{'common_include.unittest': True}
'Reading History from G:\\My Drive\\vepar\\all_history_test.csv'


In [5]:
@dataclass
class h_test(Historys):
    def __post_init__(self):
        # hist_file = rootdir + "all_history_test.csv"
        super().__post_init__()
        return

ht = h_test()

{'common_include.unittest': True}
'Reading History from G:\\My Drive\\vepar\\all_history_test.csv'


In [6]:
@dataclass
class h_test(Historys):
    def __post_init__(self):
        # hist_file = rootdir + "all_history_test.csv"
        super().__post_init__()
        return
    def prep_summ_df(self):
        # Sample Data (without 'trade_type', using quantity sign to differentiate buy/sell)
        # data = {
        #     'trade_id': [1, 2, 3, 4, 5],
        #     'symbol': ['AAPL', 'GOOG', 'AAPL', 'GOOG', 'AMZN'],
        #     'price': [150.0, 2800.0, 155.0, 2850.0, 3500.0],
        #     'quantity': [10, -5, 15, -8, 2],  # Negative quantity for sell trades
        #     'trade_date': ['2023-05-10', '2023-05-11', '2023-05-12', '2023-05-13', '2023-05-14']
        # }

        # return
        history_trades_df = self.getDF()
        # history_trades_df[Price].replace({'\,': ''})
        history_trades_df['price'] = history_trades_df[Price].replace({'\$': ''}, regex=True).replace({',': ''}, regex=True).astype(float)
        # Convert 'trade_date' to datetime for any date-based operations
        history_trades_df['trade_date'] = pd.to_datetime(history_trades_df[Date])
        trade_count = len(history_trades_df)

        # Group by 'symbol' and aggregate data
        self.summary_df = history_trades_df.groupby(Symbol).agg(
            qty_buy=(Quantity, lambda x: x[x > 0].sum()),  # Sum of positive quantities for buy trades
            qty_sell=(Quantity, lambda x: x[x < 0].sum()),  # Sum of negative quantities for sell trades
            avg_price_buy=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].mean()),
            # Average buy price
            avg_price_sell=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].mean()),
            first_trade_price=('price', 'last'),  # Last trade price
            last_trade_price=('price', 'first'),
            last_trade_qty=(Quantity, 'first'),
            # total_buy_amt=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] > 0].sum()),
            # # Total buy amount
            # total_sell_amt=('price', lambda x: x[history_trades_df.loc[x.index, Quantity] < 0].sum()),
            # Average sell price
            # trade_count=('trade_id', 'count'),  # Count of trades
            first_trade_date=('trade_date', 'min'),  # First trade date
            last_trade_date=('trade_date', 'max')  # Last trade date
        ).reset_index()